In [1]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from tqdm import tqdm

# --- Load and Preprocess Data (DO NOT MODIFY AFTER THIS) ---
ndf = pd.read_csv("../Downloads/st_exportd.csv").iloc[:,1:]
for i in range(len(ndf)):
    val = ndf.iloc[i,-3]
    if isinstance(val, str):
        ndf.iloc[i,-3] = val.split("GMT")[0].strip()
    else:
        ndf.iloc[i,-3] = np.nan
ndf['timestamp'] = pd.to_datetime(ndf['timestamp'], errors='coerce')
ndf = ndf.sort_values('timestamp').reset_index(drop=True)

In [2]:
import datetime

In [3]:
# --- Display Results (FULL OUTPUT) ---
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect terminal width
pd.set_option('display.max_colwidth', None)  # Show full content of each column

In [4]:
len(ndf)

140060

In [10]:
ndf[ndf["timestamp"]>"2025-12-19 23:49:50"].head(25)

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
138800,10681630,-1.838902e+06,-53.147423,-0.711714,231.050522,-0.014501,1.531474,3.312423,0.007700,-9.553489,-1.000000,TAO,2025-12-19 23:49:59,3m,100
138801,409176896,8.891169e+06,53.415798,0.837373,88260.328125,-0.003691,0.465842,3.827168,0.002441,-2.696297,-0.990941,BTC,2025-12-19 23:49:59,3m,100
138802,355599904,-2.926715e+05,-6.458274,-0.910973,2974.658203,-0.003106,1.064583,4.249540,0.003236,-4.341850,-0.999661,ETH,2025-12-19 23:49:59,3m,100
138803,98266136,2.305342e+06,10.980186,0.834199,125.847755,-0.004194,0.890198,2.326041,0.003969,-2.187758,-0.975149,SOL,2025-12-19 23:49:59,3m,100
138804,28765152,5.364781e+05,12.210883,0.843361,429.038879,0.001098,0.877891,0.517605,0.013876,0.035960,0.035944,ZEC,2025-12-19 23:49:59,3m,100
138805,49824564,-1.501140e+05,-2.401256,-0.891506,1.877606,-0.000376,1.024013,4.557215,0.003476,-0.505026,-0.466060,XRP,2025-12-19 23:49:59,3m,100
138806,36745344,3.130952e+05,6.960959,0.868315,845.990295,-0.000473,0.930390,0.168611,0.001760,-0.042164,-0.042139,BNB,2025-12-19 23:49:59,3m,100
138807,10636919,-1.733691e+06,-38.276470,-0.716629,12.128851,0.003393,1.382765,4.459547,0.007124,2.936696,0.994389,AVAX,2025-12-19 23:49:59,3m,100
138808,13108416,-4.003312e+05,-15.255272,-0.825107,1.444300,-0.005539,1.152553,2.299269,0.007107,-2.065211,-0.968356,SUI,2025-12-19 23:49:59,3m,100
138809,422186368,-2.261459e+06,-15.650717,-0.879953,88214.351562,-0.003403,1.156507,4.006634,0.002359,-6.684742,-0.999997,BTC,2025-12-20 00:01:17,1m,300


# predict e

In [11]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

# --- Split Data (DO NOT MODIFY AFTER THIS) ---
train_df = ndf.iloc[:138809, :].copy()
actual_x = ndf.iloc[138809:, :].copy()

# --- Define Gap Groups and Target ---
short_gaps = ['1m', '3m', '5m', '15m', '30m', '1h']
long_gaps = ['2h', '4h', '6h', '8h', '12h']
target = 'e'  # Predict 'e' instead of 'h'
features = ['klineacc', 'spread', 'vwap', 'deviation', 'ratio', 'term', 'sigma']

# --- Helper Functions ---
def get_last_train_timestamp(df, asset, gap):
    """Return the last timestamp for a given asset and gap in train_df."""
    subset = df[(df['asset'] == asset) & (df['gap'] == gap)]
    return subset['timestamp'].max()

def shortlist_actual_x(actual_x, asset, gap, end_duration):
    """Shortlist actual_x entries for evaluation."""
    mask = (
        (actual_x['asset'] == asset) &
        (actual_x['gap'] == gap) &
        (actual_x['timestamp'] <= end_duration)
    )
    return actual_x[mask]

# --- Training ---
models = {}
unique_assets = set()
for key in train_df['asset'].unique():
    unique_assets.add(key)

for asset in tqdm(unique_assets, desc="Training assets"):
    for gap in short_gaps + long_gaps:
        # Filter train_df for this asset and gap
        train_subset = train_df[(train_df['asset'] == asset) & (train_df['gap'] == gap)]
        if len(train_subset) == 0:
            continue

        # Prepare data
        X_train = train_subset[features]
        y_train = train_subset[target]

        # Train XGBoost model
        model = XGBRegressor(
            objective='reg:squarederror',
            n_estimators=200,
            max_depth=6,
            learning_rate=0.05,
            reg_lambda=1,  # L2 regularization to prevent overfitting
            random_state=42
        )
        model.fit(X_train, y_train)

        # Save model
        model_key = f"{asset}_{gap}"
        models[model_key] = model

# --- Save All Models to Disk ---
with open('trained_models_e.pkl', 'wb') as f:
    pickle.dump(models, f)

print("Training complete. Models saved to 'trained_models_e.pkl'.")

Training assets: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:49<00:00,  1.49it/s]


Training complete. Models saved to 'trained_models_e.pkl'.


In [12]:
# --- Evaluation ---
results = []
predicted_results = []  # Store predicted vs actual 'e' and 'h'

# Load models
with open('trained_models_e.pkl', 'rb') as f:
    models = pickle.load(f)

print(unique_assets,len(unique_assets))

for asset in tqdm(unique_assets, desc="Evaluating assets"):
    for gap in short_gaps + long_gaps:
        model_key = f"{asset}_{gap}"
        if model_key not in models:
            continue

        # Get last train timestamp
        last_train_ts = get_last_train_timestamp(train_df, asset, gap)
        if pd.isna(last_train_ts):
            continue

        # Calculate end_duration
        if gap in short_gaps:
            duration_minutes = 300
        else:
            duration_minutes = 7200
        end_duration = last_train_ts + pd.Timedelta(minutes=duration_minutes)

        # Shortlist actual_x
        eval_subset = shortlist_actual_x(actual_x, asset, gap, end_duration)
        if len(eval_subset) == 0:
            continue

        # Prepare data
        X_eval = eval_subset[features]
        y_eval_e = eval_subset[target]  # Actual 'e'
        y_eval_h = eval_subset['h']    # Actual 'h' (for directional accuracy)

        # Predict 'e'
        model = models[model_key]
        y_pred_e = model.predict(X_eval)
        y_pred_h = np.tanh(y_pred_e)    # Compute 'h' from predicted 'e'

        # Calculate metrics
        mae_e = mean_absolute_error(y_eval_e, y_pred_e)
        mae_h = mean_absolute_error(y_eval_h, y_pred_h)
        directional_acc = np.mean((np.sign(y_eval_h) == np.sign(y_pred_h)).astype(float))

        # Store evaluation results
        results.append({
            'asset': asset,
            'gap': gap,
            'last_train_timestamp': last_train_ts,
            'end_duration': end_duration,
            'n_eval_samples': len(eval_subset),
            'MAE_e': mae_e,
            'MAE_h': mae_h,
            'Directional_Accuracy': directional_acc
        })

        # Store predicted vs actual 'e' and 'h'
        eval_subset = eval_subset.reset_index(drop=True)
        for idx in range(len(eval_subset)):
            predicted_results.append({
                'asset': asset,
                'gap': gap,
                'timestamp': eval_subset.loc[idx, 'timestamp'],
                'actual_e': eval_subset.loc[idx, target],
                'predicted_e': y_pred_e[idx],
                'actual_h': eval_subset.loc[idx, 'h'],
                'predicted_h': y_pred_h[idx],
                'absolute_error_e': abs(eval_subset.loc[idx, target] - y_pred_e[idx]),
                'absolute_error_h': abs(eval_subset.loc[idx, 'h'] - y_pred_h[idx]),
                'last_train_timestamp': last_train_ts
            })

# --- Save Results ---
results_df = pd.DataFrame(results)
predicted_df = pd.DataFrame(predicted_results)

# Find the entry with the biggest error for 'e' and 'h'
biggest_error_e_df = predicted_df.loc[predicted_df.groupby(['asset', 'gap'])['absolute_error_e'].idxmax()]
biggest_error_h_df = predicted_df.loc[predicted_df.groupby(['asset', 'gap'])['absolute_error_h'].idxmax()]

# Save to CSV
results_df.to_csv('evaluation_results_e.csv', index=False)
predicted_df.to_csv('predicted_vs_actual_e_h.csv', index=False)
biggest_error_e_df.to_csv('biggest_error_e_entries.csv', index=False)
biggest_error_h_df.to_csv('biggest_error_h_entries.csv', index=False)

print("Evaluation complete. Results saved to:")
print("- evaluation_results_e.csv (summary)")
print("- predicted_vs_actual_e_h.csv (all predicted vs actual 'e' and 'h' values)")
print("- biggest_error_e_entries.csv (entries with the biggest error for 'e')")
print("- biggest_error_h_entries.csv (entries with the biggest error for 'h')")

{'PAXG', 'DOT', 'RENDER', 'BNSOL', 'PYR', 'ICP', 'SSV', 'NEO', 'XRP', 'ORDI', 'CAKE', 'TON', 'ENS', 'ZEN', 'TRB', 'ZRO', 'TWT', 'BANANA', 'MOVR', 'APT', 'ASTER', 'WBTC', 'RAY', 'INJ', 'PENDLE', 'EUL', 'XNO', 'ZEC', 'DASH', 'NMR', 'AAVE', 'FIL', 'MORPHO', 'ASR', 'ILV', 'SOL', 'AXS', 'COMP', 'DCR', 'LPT', 'GIGGLE', 'AUCTION', 'AVAX', 'BNB', 'FARM', 'EGLD', 'LTC', 'TRUMP', 'TAO', 'WBETH', '0G', 'ALCX', 'METIS', 'KSM', 'SANTOS', 'MMT', 'UNI', 'LINK', 'BCH', 'BERA', 'ETC', 'BTC', 'ETH', 'VANA', 'QNT', 'MLN', 'ORCA', 'SUI', 'ENSO', 'ATOM', 'OG', 'NEAR', 'VIRTUAL', 'AR'} 74


Evaluating assets: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:11<00:00,  6.64it/s]


Evaluation complete. Results saved to:
- evaluation_results_e.csv (summary)
- predicted_vs_actual_e_h.csv (all predicted vs actual 'e' and 'h' values)
- biggest_error_e_entries.csv (entries with the biggest error for 'e')
- biggest_error_h_entries.csv (entries with the biggest error for 'h')


In [13]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

# --- Load and Preprocess Data (DO NOT MODIFY) ---
# (Assume ndf, train_df, actual_x are already loaded and preprocessed)

# --- Load Trained Models ---
with open('trained_models_e.pkl', 'rb') as f:
    models = pickle.load(f)

# --- Define Gap Groups and Target ---
short_gaps = ['1m', '3m', '5m', '15m', '30m', '1h']
long_gaps = ['2h', '4h', '6h', '8h', '12h']
target = 'e'  # Predict 'e' and compute 'h'
features = ['klineacc', 'spread', 'vwap', 'deviation', 'ratio', 'term', 'sigma']

# --- Helper Functions ---
def get_last_train_timestamp(df, asset, gap):
    """Return the last timestamp for a given asset and gap in train_df."""
    subset = df[(df['asset'] == asset) & (df['gap'] == gap)]
    return subset['timestamp'].max()

def shortlist_actual_x(actual_x, asset, gap, end_duration):
    """Shortlist actual_x entries for evaluation."""
    mask = (
        (actual_x['asset'] == asset) &
        (actual_x['gap'] == gap) &
        (actual_x['timestamp'] <= end_duration)
    )
    return actual_x[mask]

# --- Evaluation ---
results = []
predicted_results = []  # Store ALL predicted vs actual 'e' and 'h' for EVERY ROW

# Extract unique assets from model keys
unique_assets = set()
for key in models.keys():
    asset = key.split('_')[0]
    unique_assets.add(asset)

for asset in tqdm(unique_assets, desc="Evaluating assets"):
    for gap in short_gaps + long_gaps:
        model_key = f"{asset}_{gap}"
        if model_key not in models:
            continue

        # Get last train timestamp
        last_train_ts = get_last_train_timestamp(train_df, asset, gap)
        if pd.isna(last_train_ts):
            continue

        # Calculate end_duration
        if gap in short_gaps:
            duration_minutes = 300
        else:
            duration_minutes = 7200
        end_duration = last_train_ts + pd.Timedelta(minutes=duration_minutes)

        # Shortlist actual_x
        eval_subset = shortlist_actual_x(actual_x, asset, gap, end_duration)
        if len(eval_subset) == 0:
            continue

        # Prepare data
        X_eval = eval_subset[features]
        y_eval_e = eval_subset[target]  # Actual 'e'
        y_eval_h = eval_subset['h']    # Actual 'h'

        # Predict 'e' and compute 'h'
        model = models[model_key]
        y_pred_e = model.predict(X_eval)
        y_pred_h = np.tanh(y_pred_e)  # Compute 'h' from predicted 'e'

        # Calculate metrics (for summary)
        mae_e = mean_absolute_error(y_eval_e, y_pred_e)
        mae_h = mean_absolute_error(y_eval_h, y_pred_h)
        directional_acc = np.mean((np.sign(y_eval_h) == np.sign(y_pred_h)).astype(float))

        # Store evaluation results (summary)
        results.append({
            'asset': asset,
            'gap': gap,
            'last_train_timestamp': last_train_ts,
            'end_duration': end_duration,
            'n_eval_samples': len(eval_subset),
            'MAE_e': mae_e,
            'MAE_h': mae_h,
            'Directional_Accuracy': directional_acc
        })

        # Store predicted vs actual 'e' and 'h' for EVERY ROW in eval_subset
        eval_subset = eval_subset.reset_index(drop=True)
        for idx in range(len(eval_subset)):
            predicted_results.append({
                'asset': asset,
                'gap': gap,
                'timestamp': eval_subset.loc[idx, 'timestamp'],
                'actual_e': eval_subset.loc[idx, target],
                'predicted_e': y_pred_e[idx],
                'actual_h': eval_subset.loc[idx, 'h'],
                'predicted_h': y_pred_h[idx],
                'absolute_error_e': abs(eval_subset.loc[idx, target] - y_pred_e[idx]),
                'absolute_error_h': abs(eval_subset.loc[idx, 'h'] - y_pred_h[idx]),
                'last_train_timestamp': last_train_ts
            })

# --- Save Results ---
results_df = pd.DataFrame(results)
predicted_df = pd.DataFrame(predicted_results)

# Save to CSV
results_df.to_csv('evaluation_results_e.csv', index=False)
predicted_df.to_csv('predicted_vs_actual_e_h_detailed.csv', index=False)  # ALL ROWS

print("Evaluation complete. Results saved to:")
print("- evaluation_results_e.csv (summary metrics)")
print("- predicted_vs_actual_e_h_detailed.csv (EVERY predicted vs actual 'e' and 'h' for all rows)")

Evaluating assets: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:11<00:00,  6.49it/s]


Evaluation complete. Results saved to:
- evaluation_results_e.csv (summary metrics)
- predicted_vs_actual_e_h_detailed.csv (EVERY predicted vs actual 'e' and 'h' for all rows)


In [14]:
f=['BTC','ETH','SOL','ZEC','BNB','XRP','PAXG','BCH','AAVE','TAO']

In [15]:
results_df.sort_values(['asset', 'gap'])[results_df["asset"].isin(f)]

/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_97053/944919742.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results_df.sort_values(['asset', 'gap'])[results_df["asset"].isin(f)]


,asset,gap,last_train_timestamp,end_duration,n_eval_samples,MAE_e,MAE_h,Directional_Accuracy
40,AAVE,2h,2025-12-19 20:10:38,2025-12-24 20:10:38,3,0.043321,0.042187,1.000000
41,AAVE,4h,2025-12-19 18:22:53,2025-12-24 18:22:53,1,0.082842,0.081477,1.000000
79,BCH,2h,2025-12-19 20:10:38,2025-12-24 20:10:38,3,0.192821,0.032463,1.000000
80,BCH,4h,2025-12-19 18:22:53,2025-12-24 18:22:53,1,0.021933,0.002500,1.000000
62,BNB,15m,2025-12-19 22:58:05,2025-12-20 03:58:05,1,0.033063,0.024125,1.000000
63,BNB,1h,2025-12-19 23:42:05,2025-12-20 04:42:05,1,0.051190,0.036356,1.000000
59,BNB,1m,2025-12-19 23:48:54,2025-12-20 04:48:54,23,0.117753,0.038946,0.956522
64,BNB,2h,2025-12-19 20:10:38,2025-12-24 20:10:38,3,0.587308,0.415626,1.000000
60,BNB,3m,2025-12-19 23:49:59,2025-12-20 04:49:59,13,0.098655,0.055353,1.000000
65,BNB,4h,2025-12-19 18:22:53,2025-12-24 18:22:53,1,0.831595,0.612538,1.000000


In [16]:
predicted_df.sort_values(['asset', 'gap'])[predicted_df["asset"].isin(f)]

/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_97053/1472433255.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  predicted_df.sort_values(['asset', 'gap'])[predicted_df["asset"].isin(f)]


,asset,gap,timestamp,actual_e,predicted_e,actual_h,predicted_h,absolute_error_e,absolute_error_h,last_train_timestamp
166,AAVE,2h,2025-12-20 02:13:58,-0.097332,-0.206001,-0.097025,-0.203135,0.108669,0.106110,2025-12-19 20:10:38
167,AAVE,2h,2025-12-20 08:17:20,-0.221783,-0.203427,-0.218217,-0.200667,0.018356,0.017550,2025-12-19 20:10:38
168,AAVE,2h,2025-12-20 14:20:37,-0.114873,-0.111935,-0.114371,-0.111470,0.002938,0.002900,2025-12-19 20:10:38
169,AAVE,4h,2025-12-20 06:26:08,0.085520,0.168362,0.085312,0.166789,0.082842,0.081477,2025-12-19 18:22:53
349,BCH,2h,2025-12-20 02:13:57,2.166276,2.133208,0.974073,0.972324,0.033067,0.001748,2025-12-19 20:10:38
350,BCH,2h,2025-12-20 08:17:20,1.921618,2.081398,0.958050,0.969349,0.159780,0.011299,2025-12-19 20:10:38
351,BCH,2h,2025-12-20 14:20:37,1.209888,1.595502,0.836646,0.920989,0.385615,0.084343,2025-12-19 20:10:38
352,BCH,4h,2025-12-20 06:26:08,1.738318,1.760252,0.940031,0.942531,0.021933,0.002500,2025-12-19 18:22:53
314,BNB,15m,2025-12-20 03:33:14,0.592767,0.559705,0.531883,0.507758,0.033063,0.024125,2025-12-19 22:58:05
315,BNB,1h,2025-12-20 02:45:30,0.576164,0.627354,0.519872,0.556227,0.051190,0.036356,2025-12-19 23:42:05


In [17]:
biggest_error_e_df.sort_values(['asset', 'gap'])[biggest_error_e_df["asset"].isin(f)]

,asset,gap,timestamp,actual_e,predicted_e,actual_h,predicted_h,absolute_error_e,absolute_error_h,last_train_timestamp
166,AAVE,2h,2025-12-20 02:13:58,-0.097332,-0.206001,-0.097025,-0.203135,0.108669,0.106110,2025-12-19 20:10:38
169,AAVE,4h,2025-12-20 06:26:08,0.085520,0.168362,0.085312,0.166789,0.082842,0.081477,2025-12-19 18:22:53
351,BCH,2h,2025-12-20 14:20:37,1.209888,1.595502,0.836646,0.920989,0.385615,0.084343,2025-12-19 20:10:38
352,BCH,4h,2025-12-20 06:26:08,1.738318,1.760252,0.940031,0.942531,0.021933,0.002500,2025-12-19 18:22:53
314,BNB,15m,2025-12-20 03:33:14,0.592767,0.559705,0.531883,0.507758,0.033063,0.024125,2025-12-19 22:58:05
315,BNB,1h,2025-12-20 02:45:30,0.576164,0.627354,0.519872,0.556227,0.051190,0.036356,2025-12-19 23:42:05
271,BNB,1m,2025-12-20 01:51:56,-1.204852,-1.637733,-0.835129,-0.927155,0.432881,0.092026,2025-12-19 23:48:54
317,BNB,2h,2025-12-20 08:17:20,1.188138,0.197355,0.830001,0.194832,0.990783,0.635168,2025-12-19 20:10:38
288,BNB,3m,2025-12-20 00:54:21,0.739817,0.998940,0.629035,0.761149,0.259123,0.132114,2025-12-19 23:49:59
319,BNB,4h,2025-12-20 06:26:08,0.967483,0.135888,0.747596,0.135058,0.831595,0.612538,2025-12-19 18:22:53


In [18]:
biggest_error_h_df.sort_values(['absolute_error_h'])[biggest_error_h_df["asset"].isin(f)]

/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_97053/3434181425.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  biggest_error_h_df.sort_values(['absolute_error_h'])[biggest_error_h_df["asset"].isin(f)]


,asset,gap,timestamp,actual_e,predicted_e,actual_h,predicted_h,absolute_error_e,absolute_error_h,last_train_timestamp
352,BCH,4h,2025-12-20 06:26:08,1.738318,1.760252,0.940031,0.942531,0.021933,0.002500,2025-12-19 18:22:53
156,ZEC,15m,2025-12-20 03:33:14,0.810206,0.798523,0.669704,0.663210,0.011683,0.006493,2025-12-19 22:58:05
314,BNB,15m,2025-12-20 03:33:14,0.592767,0.559705,0.531883,0.507758,0.033063,0.024125,2025-12-19 22:58:05
409,BTC,1h,2025-12-20 02:45:30,-1.727910,-2.035846,-0.938809,-0.966474,0.307935,0.027666,2025-12-19 23:42:05
315,BNB,1h,2025-12-20 02:45:30,0.576164,0.627354,0.519872,0.556227,0.051190,0.036356,2025-12-19 23:42:05
3,PAXG,4h,2025-12-20 06:26:08,0.300243,0.254773,0.291535,0.249400,0.045470,0.042135,2025-12-19 18:22:53
226,SOL,1h,2025-12-20 02:45:30,-1.359033,-1.171371,-0.876169,-0.824711,0.187662,0.051457,2025-12-19 23:42:05
230,SOL,4h,2025-12-20 06:26:08,0.524487,0.459259,0.481156,0.429480,0.065228,0.051676,2025-12-19 18:22:53
71,XRP,2h,2025-12-20 14:20:37,0.363601,0.304613,0.348382,0.295529,0.058988,0.052853,2025-12-19 20:10:38
336,TAO,5m,2025-12-20 00:22:15,-4.069548,-1.783249,-0.999416,-0.945044,2.286299,0.054373,2025-12-19 23:45:33
